Import first

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sci
import scipy.stats as stats
import statsmodels.api as sm
import pandas as pd
from scipy.optimize import minimize

Let's create the variables first

In [3]:
s=1000
x1=stats.poisson(20).rvs(s)
x2=stats.norm(10, 3).rvs(s)
x0=np.ones((s, 1))
y=5+3*x1-x2+stats.norm(0, 1).rvs(s)

In [4]:
X=np.column_stack((x0, x1, x2))

In [5]:
# let's see the matrix method first
betahat=np.linalg.inv(X.T@X)@X.T@y

In [6]:
betahat

array([ 4.84242581,  3.00734506, -0.99977758])

In [ ]:
#results are pretty close to our created parameters.

In [ ]:
# let's take these to Pandas

In [7]:
data=pd.DataFrame(X, columns=['x0', 'x1', 'x2'])

In [8]:
data.head()

,x0,x1,x2
0,1.0,20.0,7.515461
1,1.0,21.0,12.100347
2,1.0,25.0,9.683521
3,1.0,19.0,9.764315
4,1.0,20.0,9.951834


In [9]:
data['y']=y

In [10]:
data.head()

,x0,x1,x2,y
0,1.0,20.0,7.515461,59.058003
1,1.0,21.0,12.100347,56.306277
2,1.0,25.0,9.683521,69.932572
3,1.0,19.0,9.764315,51.798584
4,1.0,20.0,9.951834,54.011988


Next, create the sum of squared errors function, this actually is the objective function for least squares

In [11]:
def sse(theta):
    data['error']=data['y']-theta[0]-theta[1]*data['x1']-theta[2]*data['x2']
    sqerr=data['error']*data['error']
    sse=sqerr.sum()
    return sse

Check with off the shelve optimizer first

In [13]:
init=[1, 1, 1]
res=minimize(sse, init, method='nelder-mead')

In [14]:
res.x # results are again pretty close.

array([ 4.8424771 ,  3.00734318, -0.99977932])

In [15]:
# next, let's try gradient descent

In [16]:
# style 1, not using the pandas dataframe

In [17]:
# the sse function, just in vector/array forms
def objfn(theta):
    e=y-theta[0]-theta[1]*x1-theta[2]*x2
    return e.T@e

In [18]:
# this is the hypothesis function, can estimate other models by changing this function
def predi(theta):
    return X@theta

First, if we run the iteration for a certain number of times

In [28]:
thetaold=np.asarray([0, 0, 0])
count=0
while count<100001:
    thetanew=thetaold+0.000001*(X.T@(y-predi(thetaold)))
    thetaold=thetanew
    
    count=count+1

In [29]:
thetanew # close to the true parameter values.

array([ 4.56687959,  3.0163859 , -0.99116203])

Instead of running the loop for certain number of iterations, let's try for a convergence condition

In [32]:
thetaold=np.asarray([0, 0, 0])
thetanew=np.asarray([10, 10, 10])
prevtheta=thetaold
while abs(objfn(thetanew)-objfn(prevtheta))>0.0001:
    
    thetanew=thetaold+0.000001*(X.T@(y-predi(thetaold)))
    prevtheta=thetaold
    thetaold=thetanew
    

In [33]:
thetanew # close numbers again

array([ 4.59460768,  3.01547612, -0.99202901])

In [34]:
# the same with pandas dataframe

In [36]:
count=0
thetaold=[0, 0, 0]
while count<100001:
    error=data['y']-thetaold[0]-thetaold[1]*data['x1']-thetaold[2]*data['x2']
    thetaold[0]=thetaold[0]+0.000001*error@x0
    thetaold[1]=thetaold[1]+0.000001*error@x1
    thetaold[2]=thetaold[2]+0.000001*error@x2
    count=count+1

In [37]:
thetaold # compare with previous results

[array([4.56687959]), 3.016385896205776, -0.9911620286836779]